In [1]:
file = "../dataset/data-nottag.set"

In [2]:
import pandas as pd
colnames=['text', 'tag']
def filtered_words(x:tuple):
    #w = .lower()#word_tokenize(x[0],custom_dict=o)
    #ww = [word for word in w if word not in list(thai_stopwords())]
    return (x[0].lower().strip() ,x[1])#(''.join(ww),x[1])

In [3]:
user1 = pd.read_csv(file, names=colnames, header=None,sep="|")

In [4]:
user1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 2 columns):
text    423 non-null object
tag     423 non-null object
dtypes: object(2)
memory usage: 6.7+ KB


In [5]:
from typing import List
from sklearn.pipeline import Pipeline
from pythainlp.ulmfit import process_thai
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
X = user1.text
y = user1.tag

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.3, random_state=42)#,shuffle=True)

In [8]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [9]:
f1=[]
report=[]
for model in [MultinomialNB(),
              BernoulliNB(),
              DecisionTreeClassifier(),
              RandomForestClassifier(criterion='entropy'),
              SVC(),
              MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(24, 6), verbose=False, max_iter=2000)
             ]:
    nb = Pipeline([('vect', CountVectorizer(tokenizer=process_thai, ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('chi2', SelectKBest(chi2, k='all')),
               ('clf', model),
    ])
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    t = accuracy_score(y_pred, y_test)
    del nb
    print(t)
    f1.append(t)
    tt=classification_report(y_test, y_pred)
    #print(tt)
    report.append(tt)

0.889763779527559
0.5905511811023622


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.7874015748031497


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8740157480314961


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.3543307086614173


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.968503937007874


In [10]:
print(report[-1])

              precision    recall  f1-score   support

       alert       0.98      1.00      0.99        45
     asktime       1.00      1.00      1.00         8
         fan       1.00      1.00      1.00         4
       light       1.00      1.00      1.00         8
       music       1.00      0.86      0.92        14
        news       1.00      1.00      1.00         9
    religion       0.78      1.00      0.88         7
         sos       0.94      1.00      0.97        15
     weather       1.00      0.88      0.94        17

    accuracy                           0.97       127
   macro avg       0.97      0.97      0.97       127
weighted avg       0.97      0.97      0.97       127



In [11]:
print(report[0])

              precision    recall  f1-score   support

       alert       0.94      1.00      0.97        45
     asktime       0.88      0.88      0.88         8
         fan       1.00      0.75      0.86         4
       light       1.00      0.50      0.67         8
       music       0.63      0.86      0.73        14
        news       1.00      0.56      0.71         9
    religion       0.70      1.00      0.82         7
         sos       1.00      1.00      1.00        15
     weather       1.00      0.88      0.94        17

    accuracy                           0.89       127
   macro avg       0.90      0.82      0.84       127
weighted avg       0.91      0.89      0.89       127



In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(24, 6), verbose=False, max_iter=2000)

In [11]:
nb = Pipeline([('vect', CountVectorizer(tokenizer=process_thai, ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('chi2', SelectKBest(chi2, k='all')),
               ('clf', model),
])

In [12]:
nb.fit(X, y)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function process_th...
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(24, 6),
                               learning_rate='constant',
                               learning_rate_init=0.001, max_iter=2000,
    

In [14]:
import dill
with open('../../modelclass2.model', 'wb') as out_strm: 
    dill.dump(nb, out_strm)